# RAW DATA TO MONGODB

In [1]:
import pymongo
from pymongo import MongoClient
import pprint
import pandas as pd
import json as json
import numpy as np
#import findspark
from datetime import datetime
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.\
builder.\
appName("pyspark-notebook2").\
config("spark.executor.memory", "1g").\
config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()


In [3]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN

gpo = db.GPO
product = db.PRODUCT
hospital = db.HOSPITAL
user_roles = db.USER_ROLES
distributor = db.DISTRIBUTOR
manufacturer = db.MANUFACTURER
correlation = db.CORRELATION_DATA

In [98]:
print("spark session and mongodb connect")

spark session and mongodb connect


# Read all manufacturer catalog data

In [5]:
manufacturer_catalog1=spark.read.csv("CSV-data/CSV/product catalog/nova_product_ctalog.csv", header = True).dropna(how='all')

In [6]:
#manufacturer_catalog1.columns

In [7]:
manufacturer_catalog1=manufacturer_catalog1.withColumnRenamed('base _price','base_price')

In [8]:
manufacturer_catalog1= manufacturer_catalog1.select(col('manufacturer'),col('manufacturer_catalog_number'),
                                                    col('item_number'), col('description'), col('size'), 
                                                    col('gauge'),col('capacity'), col('color'),col('BUOM'),
                                                    col('UM'),col('base_price'), col('type_id'), col('name'),
                                                    col('short_description'),col('status'), col('is_pack'),
                                                    col('units_in_pack'),col('category'),
                                                    col('mfg_date'), col('expiry_date'), col('image'))

In [9]:
#manufacturer_catalog1.columns

In [10]:
manufacturer_catalog2=spark.read.csv("CSV-data/CSV/product catalog/Medical_action_product_manu.csv", header = True).dropna(how='all')

In [11]:
#manufacturer_catalog2.columns

In [12]:
manufacturer_catalog2= manufacturer_catalog2.select(col('manufacturer'),col('manufacturer_catalog_number'),
                                                    col('item_number'), col('description'), col('size'), 
                                                    col('gauge'),col('capacity'), col('color'),col('BUOM'),
                                                    col('UM'),col('base_price'), col('type_id'), col('name'),
                                                    col('short_description'),col('status'), col('is_pack'),
                                                    col('units_in_pack'),col('category'),
                                                    col('mfg_date'), col('expiry_date'), col('image'))

In [13]:
#manufacturer_catalog2.columns

In [14]:
manufacturer_catalog3=spark.read.csv("CSV-data/CSV/product catalog/MALLINCKRODT_product_catalog.csv", header = True).dropna(how='all')

In [15]:
#manufacturer_catalog3.columns

In [16]:
manufacturer_catalog3 = manufacturer_catalog3.withColumnRenamed('is_pack ','is_pack')

In [17]:
manufacturer_catalog3= manufacturer_catalog3.select(col('manufacturer'),col('manufacturer_catalog_number'),
                                                    col('item_number'), col('description'), col('size'), 
                                                    col('gauge'),col('capacity'), col('color'),col('BUOM'),
                                                    col('UM'),col('base_price'), col('type_id'), col('name'),
                                                    col('short_description'),col('status'), col('is_pack'),
                                                    col('units_in_pack'),col('category'),
                                                    col('mfg_date'), col('expiry_date'), col('image'))

In [18]:
#manufacturer_catalog3.columns

## column name are the same so now perform union

In [19]:
from pyspark.sql import DataFrame

In [20]:
manufacturer_catalog = manufacturer_catalog3.union(manufacturer_catalog1.union(manufacturer_catalog2))

In [21]:
#just to see in clear format we use .Pandas() method
#manufacturer_catalog.toPandas()

In [22]:
#manufacturer_catalog.columns

## Lets rename the column name

In [23]:
manufacturer_catalog=manufacturer_catalog.withColumnRenamed('item_number','PRODUCT_ID').\
                                          withColumnRenamed('description','DESCRIPTION').\
                                          withColumnRenamed('manufacturer','MANUFACTURER').\
                                          withColumnRenamed('size','SIZE').\
                                          withColumnRenamed('manufacturer_catalog_number','CATALOG_NUMBER').\
                                          withColumnRenamed('gauge','VOLUME').\
                                          withColumnRenamed('color','COLOR').\
                                          withColumnRenamed('BUOM','WEIGHT_UOM').\
                                          withColumnRenamed('UM','UOM').\
                                          withColumnRenamed('base_price','BASE_PRICE').\
                                          withColumnRenamed('type_id','PRODUCT_TYPE_ID').\
                                          withColumnRenamed('name','NAME').\
                                          withColumnRenamed('capacity','CAPACITY').\
                                        withColumnRenamed('short_description','SHORT_DESCRIPTION').\
                                        withColumnRenamed('status','PRODUCT_STATUS').\
                                        withColumnRenamed('is_pack','IS_PACK').\
                                        withColumnRenamed('units_in_pack','UNITS_IN_PACK').\
                                        withColumnRenamed('category','PRODUCT_CATEGORY').\
                                        withColumnRenamed('mfg_date','MFG_DATE').\
                                        withColumnRenamed('expiry_date','EXPIRY_DATE').\
                                        withColumnRenamed('image','PRODUCT_IMAGE')
                                        

In [99]:
print("manufacturer csv file read")

manufacturer csv file read


## Make MED_EX_CHAIN database in mongodb, create a collection name.

In [26]:
product.insert_one({'PRODUCT_ID': 0, 'VERSION': 0})

In [27]:
product_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
product_data.createOrReplaceTempView('product_view')

In [28]:
for i in manufacturer_catalog.toJSON().collect():
    new_record = json.loads(i)
    product_id = new_record['PRODUCT_ID']

    query = "select * from product_view \
             where PRODUCT_ID = '" + product_id + "' and \
                   VERSION = (select max(VERSION) from product_view \
             where PRODUCT_ID = '" + product_id + "')"
    
    available_data = spark.sql(query)
    print(product_id, available_data.count())
    
    # no data available, insert with version 1.0
    if available_data.count() == 0: 
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        product.insert_one(insert_data)
        print('insert data with 1 version.')
    
    # insert data with new version.    
    elif (new_record['NAME'] != available_data.rdd.collect()[0]['NAME'])|\
         (new_record['DESCRIPTION'] != available_data.rdd.collect()[0]['DESCRIPTION'])|\
         (new_record['BASE_PRICE'] != available_data.rdd.collect()[0]['BASE_PRICE'])|\
         (new_record['CATALOG_NUMBER'] != available_data.rdd.collect()[0]['CATALOG_NUMBER'])|\
         (new_record['UNITS_IN_PACK'] != available_data.rdd.collect()[0]['UNITS_IN_PACK'])|\
         (new_record['SHORT_DESCRIPTION'] != available_data.rdd.collect()[0]['SHORT_DESCRIPTION']): 
         # add conditions for all the columns which are eligible for the SCD.

         product.update_many({'PRODUCT_ID': product_id,
                              'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                       'END_DATE':str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         product.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
                
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        product.update_one({'PRODUCT_ID': product_id, 
                            'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                           {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


3054 0
insert data with 1 version.
6219 0
insert data with 1 version.
6222 0
insert data with 1 version.
6223 0
insert data with 1 version.
6224 0
insert data with 1 version.
1001 0
insert data with 1 version.
1002 0
insert data with 1 version.
1003 0
insert data with 1 version.
1004 0
insert data with 1 version.
1005 0
insert data with 1 version.
2001 0
insert data with 1 version.
2002 0
insert data with 1 version.
2003 0
insert data with 1 version.
2004 0
insert data with 1 version.
2005 0
insert data with 1 version.


## Read distributor data and union distributor

In [29]:
distributor_catalog1 = spark.read.csv("CSV-data/CSV/distributor catalog/Distributor-Mckesson.csv", header = True).dropna(how='all')

In [31]:
distributor_catalog1 = distributor_catalog1.select(col("account_name"), col("supplier_name"), col("supplier_cat_number"), col("supplier_sku_number"),col("distributor_sku_number"),
                           col("sku _description"), col("hospital_item_number"), col("quantity_shipped"), col("unit_price"), 
                           col("BUM"), col("BUM_FACTOR"), col("contract_price"),col("contract_number"), col("distributor_name"))

In [32]:
#distributor_catalog1.columns

In [33]:
distributor_catalog2=spark.read.csv("CSV-data/CSV/distributor catalog/Distributor-CardinalHealth.csv", header = True).dropna(how='all')

In [34]:
#distributor_catalog2.columns

In [35]:
from pyspark.sql import DataFrame

In [36]:
distributor_catalog = distributor_catalog1.union(distributor_catalog2)

In [37]:
distributor_catalog = distributor_catalog.withColumnRenamed('account_name','HOSPITAL_NAME').\
                                          withColumnRenamed('supplier_name','MANUFACTURER_NAME').\
                                          withColumnRenamed('supplier_cat_number','SUPPLIER_CAT_NUMBER').\
                                          withColumnRenamed('supplier_sku_number','PRODUCT_ID').\
                                          withColumnRenamed('distributor_sku_number','DISTRIBUTOR_SKU').\
                                          withColumnRenamed('sku _description','DESCRIPTION').\
                                          withColumnRenamed('hospital_item_number','HOSPITAL_SKU').\
                                          withColumnRenamed('quantity_shipped','QUANTITY').\
                                          withColumnRenamed('BUM','BUM').\
                                          withColumnRenamed('BUM_FACTOR','BUM_FACTOR').\
                                          withColumnRenamed('unit_price','BASE_PRICE').\
                                          withColumnRenamed('contract_price','CONTRACT_PRICE').\
                                          withColumnRenamed('contract_number','CONTRACT_NUMBER').\
                                        withColumnRenamed('distributor_name','DISTRIBUTOR_NAME')
                                        

In [38]:
#distributor_catalog.columns

In [39]:
#distributor_catalog.toPandas()

### User-Roles Raw Staging

In [40]:
user_roles.insert_one({'USER_ID': 0, 'VERSION': 0})

In [41]:
user_csv = spark.read.csv("User.csv", header = True).dropna(how = "all")

In [42]:
user_data = spark.read.format("mongo").option("uri",
                            "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
user_data.createOrReplaceTempView('user_view')

In [43]:
for i in user_csv.toJSON().collect():
    new_record = json.loads(i)
    user_id = new_record['USER_ID']

    query = "select * from user_view \
             where USER_ID = '" + user_id + "' and\
                   VERSION = (select max(VERSION) from user_view \
             where USER_ID = '" + user_id + "')"
    
    available_data = spark.sql(query)
    print(user_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['FIRST_NAME'] != available_data.rdd.collect()[0]['FIRST_NAME'])|\
         (new_record['USER_ADDRESS'] != available_data.rdd.collect()[0]['USER_ADDRESS'])|\
         (new_record['PHONE_NUMBER'] != available_data.rdd.collect()[0]['PHONE_NUMBER'])|\
	     (new_record['EMAIL_ID'] != available_data.rdd.collect()[0]['EMAIL_ID']): 
         # add conditions for all the columns which are eligible for the SCD.

         user_roles.update_many({'USER_ID': user_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         user_roles.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 

U1001 0
insert data with 1 version.
U1002 0
insert data with 1 version.


In [95]:
hospital_catalog = spark.read.csv("CSV-data/CSV/hospital catalog/hospital_catalog.csv", header = True).dropna(how='all')

In [96]:
#hospital_catalog.columns

In [97]:
hospital_catalog = hospital_catalog.withColumnRenamed('distributor_name','DISTRIBUTOR_NAME').\
                                          withColumnRenamed('supplier_name','MANUFACTURER_NAME').\
                                          withColumnRenamed('supplier_cat_number','SUPPLIER_CAT_NUMBER').\
                                          withColumnRenamed('supplier_sku_number','PRODUCT_ID').\
                                          withColumnRenamed('distributor_sku_number','DISTRIBUTOR_SKU').\
                                          withColumnRenamed('sku _description','DESCRIPTION').\
                                          withColumnRenamed('hospital_item_number','HOSPITAL_SKU').\
                                          withColumnRenamed('quantity_shipped','QUANTITY').\
                                          withColumnRenamed('BUM','BUM').\
                                          withColumnRenamed('BUM_FACTOR','BUM_FACTOR').\
                                          withColumnRenamed('unit_price','BASE_PRICE').\
                                          withColumnRenamed('total_price','TOTAL_PRICE').\
                                        withColumnRenamed('hospital_name','HOSPITAL_NAME')

In [47]:
#hospital_catalog.toPandas()

# CORRELATION :--

In [48]:
manufacturer_catalog.createOrReplaceTempView("manufacturer")

In [49]:
distributor_catalog.createOrReplaceTempView("distributor")

In [50]:
hospital_catalog.createOrReplaceTempView("hospital")

In [51]:
#query = "select DISTINCT manufacturer.MANUFACTURER, manufacturer.PRODUCT_ID, distributor.DISTRIBUTOR_NAME, distributor.DISTRIBUTOR_SKU,  distributor.DESCRIPTION, distributor.CONTRACT_PRICE, distributor.CONTRACT_NUMBER,distributor.BASE_PRICE,   hospital.HOSPITAL_NAME, hospital.HOSPITAL_SKU from manufacturer join distributor on manufacturer.PRODUCT_ID  = distributor.PRODUCT_ID join hospital on hospital.PRODUCT_ID = distributor.PRODUCT_ID ORDER BY distributor.DISTRIBUTOR_NAME"

In [52]:
#query = "select DISTINCT manufacturer.MANUFACTURER, manufacturer.PRODUCT_ID, distributor.DISTRIBUTOR_NAME, distributor.DISTRIBUTOR_SKU,  distributor.DESCRIPTION,distributor.HOSPITAL_NAME,   hospital.HOSPITAL_NAME, hospital.HOSPITAL_SKU from manufacturer join distributor on manufacturer.PRODUCT_ID  = distributor.PRODUCT_ID join hospital on hospital.PRODUCT_ID = distributor.PRODUCT_ID and  hospital.HOSPITAL_NAME = distributor.HOSPITAL_NAME ORDER BY distributor.DISTRIBUTOR_NAME"

In [53]:
query = "select DISTINCT manufacturer.MANUFACTURER, manufacturer.PRODUCT_ID, distributor.DISTRIBUTOR_NAME,  distributor.DESCRIPTION, distributor.DISTRIBUTOR_SKU, distributor.CONTRACT_PRICE,  distributor.CONTRACT_NUMBER, distributor.BASE_PRICE ,  hospital.HOSPITAL_NAME, hospital.HOSPITAL_SKU from manufacturer join distributor on manufacturer.PRODUCT_ID  = distributor.PRODUCT_ID join hospital on hospital.PRODUCT_ID = distributor.PRODUCT_ID AND hospital.HOSPITAL_SKU = distributor.HOSPITAL_SKU and hospital.HOSPITAL_NAME = distributor.HOSPITAL_NAME ORDER BY distributor.DISTRIBUTOR_NAME"

In [54]:
correlation_data = spark.sql(query)

In [55]:
pd_correlation = correlation_data.toPandas()

In [56]:
#pd_correlation

In [57]:
correlation.insert_many(json.loads(pd_correlation.to_json(orient = 'records')))

# read raw data and write into mongodb

# distributor_Raw data

In [58]:
distributor.insert_one({'DISTRIBUTOR_ID': 0, 'VERSION': 0})

In [59]:
distributor_csv = spark.read.csv("CSV-data/CSV/Distributor/Distributor.csv", header = True).dropna(how='all')

In [60]:
#distributor_csv.toPandas()

In [61]:
distributor_data = spark.read.format("mongo").option("uri", 
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
distributor_data.createOrReplaceTempView('distributor_view')

In [62]:
for i in distributor_csv.toJSON().collect():
    new_record = json.loads(i)
    distributor_id = new_record['DISTRIBUTOR_ID']
    
    query = "select * from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "' and\
                   VERSION = (select max(VERSION) from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "')"

    available_data = spark.sql(query)
    print(distributor_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['DISTRIBUTOR_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_NAME'])|\
         (new_record['DISTRIBUTOR_ADDRESS'] != available_data.rdd.collect()[0]['DISTRIBUTOR_ADDRESS'])|\
         (new_record['DISTRIBUTOR_SHORT_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         distributor.update_many({'DISTRIBUTOR_ID': distributor_id, 
                                  'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                           'END_DATE': str(datetime.now())}})
        

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         distributor.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.             
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.update_one({'DISTRIBUTOR_ID': distributor_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                            {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


D1001 0
insert data with 1 version.
D1002 0
insert data with 1 version.
D1003 0
insert data with 1 version.
D1004 0
insert data with 1 version.
D1005 0
insert data with 1 version.
D1006 0
insert data with 1 version.
D1007 0
insert data with 1 version.
D1008 0
insert data with 1 version.
D1009 0
insert data with 1 version.
D1010 0
insert data with 1 version.


# manufacturer data

In [63]:
manufacturer.insert_one({'MANUFACTURER_ID':0, 'VERSION': 0})

In [64]:
manufacturer_csv = spark.read.csv("Manufacturer.csv", header = True).dropna(how='all')

In [65]:
manufacturer_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
manufacturer_data.createOrReplaceTempView('manufacturer_view')

In [66]:
#manufacturer_csv.toPandas()

In [67]:
for i in manufacturer_csv.toJSON().collect():
    new_record = json.loads(i)
    manufacturer_id = new_record['MANUFACTURER_ID']

    query = "select * from manufacturer_view \
             where MANUFACTURER_ID = '" + manufacturer_id + "' and \
                   VERSION = (select max(VERSION) from manufacturer_view \
             where MANUFACTURER_ID = '" + manufacturer_id + "')"
    
    available_data = spark.sql(query)
    print(manufacturer_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['MANUFACTURER_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_NAME'])|\
         (new_record['MANUFACTURER_ADDRESS'] != available_data.rdd.collect()[0]['MANUFACTURER_ADDRESS'])|\
         (new_record['MANUFACTURER_SHORT_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         manufacturer.update_many({'MANUFACTURER_ID': manufacturer_id , 
                                   'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        
         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         manufacturer.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.update_one({'MANUFACTURER_ID': manufacturer_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

M1001 0
insert data with 1 version.
M1002 0
insert data with 1 version.
M1003 0
insert data with 1 version.
M1004 0
insert data with 1 version.
M1005 0
insert data with 1 version.


In [68]:
manufacturer_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
#manufacturer_data.toPandas()                               

# hospital data

In [69]:
hospital.insert_one({'HOSPITAL_ID':0, 'VERSION': 0})

In [70]:
hospital_csv = spark.read.csv("Hospital.csv", header = True).dropna(how='all')

In [71]:
#hospital_raw.toPandas()

In [72]:
hospital_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
hospital_data.createOrReplaceTempView('hospital_view')

In [73]:
for i in hospital_csv.toJSON().collect():
    new_record = json.loads(i)
    hospital_id = new_record['HOSPITAL_ID']

    query = "select * from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "' and\
                   VERSION = (select max(VERSION) from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "')"
    
    available_data = spark.sql(query)
    print(hospital_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['HOSPITAL_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_NAME'])|\
         (new_record['HOSPITAL_ADDRESS'] != available_data.rdd.collect()[0]['HOSPITAL_ADDRESS'])|\
         (new_record['HOSPITAL_SHORT_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.
         
         hospital.update_many({'HOSPITAL_ID': hospital_id, 
                               'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                              {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                        'END_DATE': str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         hospital.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.update_one({'HOSPITAL_ID': hospital_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


H1001 0
insert data with 1 version.
H1002 0
insert data with 1 version.
H1003 0
insert data with 1 version.
H1004 0
insert data with 1 version.
H1005 0
insert data with 1 version.
H1006 0
insert data with 1 version.


# gpo data

In [74]:
gpo.insert_one({'GPO_ID':0, 'VERSION': 0})

In [75]:
gpo_csv = spark.read.csv("GPO.csv", header = True).dropna(how='all')

In [76]:
#gpo_csv.toPandas()

In [77]:
gpo_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
gpo_data.createOrReplaceTempView('gpo_view')

In [78]:
for i in gpo_csv.toJSON().collect():
    new_record = json.loads(i)
    gpo_id = new_record['GPO_ID']

    query = "select * from gpo_view \
             where GPO_ID = '" + gpo_id + "' and\
                   VERSION = (select max(VERSION) from gpo_view \
             where GPO_ID = '" + gpo_id + "')"
    
    available_data = spark.sql(query)
    print(gpo_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['GPO_NAME'] != available_data.rdd.collect()[0]['GPO_NAME'])|\
         (new_record['GPO_ADDRESS'] != available_data.rdd.collect()[0]['GPO_ADDRESS'])|\
         (new_record['GPO_SHORT_NAME'] != available_data.rdd.collect()[0]['GPO_SHORT_NAME']): 
         # add conditions for all the columns which are eligible for the SCD.

         gpo.update_many({'GPO_ID': gpo_id, 
                          'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                         {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                   'END_DATE': str(datetime.now())}})

         insert_data = dict(new_record)
         insert_data['START_DATE'] = str(datetime.now())
         insert_data['END_DATE'] = str('9999-12-31')
         insert_data['LAST_UPDATE_DT'] = str(datetime.now())
         insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
         gpo.insert_one(insert_data)
         print('insert data with updated + 1 version.')
                
    # insert data with new version.                 
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.update_one({'GPO_ID': gpo_id, 
                             'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.


G1001 0
insert data with 1 version.
G1002 0
insert data with 1 version.
G1003 0
insert data with 1 version.
G1004 0
insert data with 1 version.
G1005 0
insert data with 1 version.
G1006 0
insert data with 1 version.
G1007 0
insert data with 1 version.
G1008 0
insert data with 1 version.
G1010 0
insert data with 1 version.


## mongodb to neo4j:--

In [79]:
from py2neo import *

graph = Graph("bolt://localhost:7687", auth = ("neo4j", "shubham"))

In [80]:
df_gpo = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
df_product = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
df_hospital = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
df_user_roles = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()    
df_distributor = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
df_manufacturer = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
df_correlation_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.CORRELATION_DATA").load()
df_product_similarity = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT_SIMILARITY").load() 

In [81]:
df_gpo.createOrReplaceTempView('gpo_view')
df_user_roles.createOrReplaceTempView('user_view')
df_product.createOrReplaceTempView('product_view')
df_hospital.createOrReplaceTempView('hospital_view')
df_distributor.createOrReplaceTempView('distributor_view')
df_manufacturer.createOrReplaceTempView('manufacturer_view')
df_correlation_data.createOrReplaceTempView('correlation_view')

In [82]:
df_gpo = spark.sql('select * from gpo_view where (GPO_ID, VERSION) in \
                   (select GPO_ID, max(VERSION) from gpo_view group by GPO_ID)')

df_user_roles = spark.sql('select * from user_view where (USER_ID, VERSION) in \
                          (select USER_ID, max(VERSION) from user_view group by USER_ID)')

df_product = spark.sql('select * from product_view where (PRODUCT_ID, VERSION) in \
                     (select PRODUCT_ID, max(VERSION) from product_view group by PRODUCT_ID)')

df_hospital = spark.sql('select * from hospital_view where (HOSPITAL_ID, VERSION) in \
                        (select HOSPITAL_ID, max(VERSION) from hospital_view group by HOSPITAL_ID)')

df_distributor = spark.sql('select * from distributor_view where (DISTRIBUTOR_ID, VERSION) in \
                           (select DISTRIBUTOR_ID, max(VERSION) from distributor_view group by DISTRIBUTOR_ID)')

df_manufacturer = spark.sql('select * from manufacturer_view where (MANUFACTURER_ID, VERSION) in \
                            (select MANUFACTURER_ID, max(VERSION) from manufacturer_view group by MANUFACTURER_ID)')

## create product node

In [83]:
#df_product.columns

In [84]:
for record in df_product.rdd.collect():
    graph.run("merge (p: Product{product_id: $product_id})\
                            on create set \
                                    p.product_category = $product_category,\
                                    p.base_price = $base_price,\
                                    p.name = $name,\
                                    p.color = $color,\
                                    p.manufacturer = $manufacturer,\
                                    p.description = $description,\
                                    p.mfg_date = $mfg_date,\
                                    p.expiry_date = $expiry_date,\
                                    p.is_pack = $is_pack,\
                                    p.product_status = $product_status,\
                                    p.product_type_id = $product_type_id,\
                                    p.size = $size,\
                                    p.short_description = $short_description,\
                                    p.units_in_pack = $units_in_pack,\
                                    p.uom = $uom,\
                                    p.volume = $volume,\
                                    p.weight_uom = $weight_uom,\
                                    p.product_image = $product_image,\
                                    p.start_date = $start_date,\
                                    p.end_date = $end_date,\
                                    p.last_update_date = $last_update_date,\
                                    p.version = $version\
                            on match set\
                                    p.product_category = $product_category,\
                                    p.base_price = $base_price,\
                                    p.name = $name,\
                                    p.color = $color,\
                                    p.manufacturer = $manufacturer,\
                                    p.description = $description,\
                                    p.mfg_date = $mfg_date,\
                                    p.expiry_date = $expiry_date,\
                                    p.is_pack = $is_pack,\
                                    p.product_status = $product_status,\
                                    p.product_type_id = $product_type_id,\
                                    p.size = $size,\
                                    p.short_description = $short_description,\
                                    p.units_in_pack = $units_in_pack,\
                                    p.uom = $uom,\
                                    p.volume = $volume,\
                                    p.weight_uom = $weight_uom,\
                                    p.product_image = $product_image,\
                                    p.start_date = $start_date,\
                                    p.end_date = $end_date,\
                                    p.last_update_date = $last_update_date,\
                                    p.version = $version",
             product_id = record['PRODUCT_ID'],
             product_category = record['PRODUCT_CATEGORY'],
             base_price = record['BASE_PRICE'],
             name = record['NAME'],
             color = record['COLOR'],
             manufacturer = record['MANUFACTURER'],
             description = record['DESCRIPTION'],
             mfg_date = record['MFG_DATE'],
             expiry_date = record['EXPIRY_DATE'],
             is_pack = record['IS_PACK'],
             product_status = record['PRODUCT_STATUS'],
             product_type_id = record['PRODUCT_TYPE_ID'],
             size = record['SIZE'],
             short_description = record['SHORT_DESCRIPTION'],
             units_in_pack = record['UNITS_IN_PACK'],
             uom = record['UOM'], 
             volume = record['VOLUME'],
             weight_uom = record['WEIGHT_UOM'],
             product_image = record['PRODUCT_IMAGE'],
             start_date = record['START_DATE'],
             end_date = record['END_DATE'],
             last_update_date = record['LAST_UPDATE_DT'],
             version = record['VERSION'])

## create manufacturer

In [85]:
for record in df_manufacturer.rdd.collect():
    graph.run("merge (m: Manufacturer{manufacturer_id: $manufacturer_id})\
                                on create set\
                                      m.country_of_origin = $country_of_origin,\
                                      m.manufacturer_name = $manufacturer_name,\
                                      m.manufacturer_address = $manufacturer_address,\
                                      m.manufacturer_org_type = $manufacturer_org_type,\
                                      m.manufacturer_short_name = $manufacturer_short_name,\
                                      m.source_identifier = $source_identifier,\
                                      m.start_date = $start_date,\
                                      m.end_date = $end_date,\
                                      m.last_update_date = $last_update_date,\
                                      m.last_update_user = $last_update_user,\
                                      m.version = $version\
                                on match set\
                                      m.country_of_origin = $country_of_origin,\
                                      m.manufacturer_name = $manufacturer_name,\
                                      m.manufacturer_address = $manufacturer_address,\
                                      m.manufacturer_org_type = $manufacturer_org_type,\
                                      m.manufacturer_short_name = $manufacturer_short_name,\
                                      m.source_identifier = $source_identifier,\
                                      m.start_date = $start_date,\
                                      m.end_date = $end_date,\
                                      m.last_update_date = $last_update_date,\
                                      m.last_update_user = $last_update_user,\
                                      m.version = $version",
        manufacturer_id = record['MANUFACTURER_ID'],
        manufacturer_name = record['MANUFACTURER_NAME'],
        country_of_origin = record['COUNTRY_OF_ORIGIN'],
        manufacturer_address = record['MANUFACTURER_ADDRESS'],
        manufacturer_org_type = record['MANUFACTURER_ORG_TYPE'],
        manufacturer_short_name = record['MANUFACTURER_SHORT_NAME'],
        source_identifier = record['SOURCE_IDENTIFIER'],
        start_date = record['START_DATE'],
        end_date = record['END_DATE'],
        last_update_date = record['LAST_UPDATE_DT'],
        last_update_user = record['LAST_UPDATE_USER'],
        version = record['VERSION'])

## create distributor

In [86]:
for record in df_distributor.rdd.collect():
    graph.run("merge (d: Distributor{distributor_id: $distributor_id})\
                            on create set\
                                     d.country_of_origin = $country_of_origin,\
                                     d.distributor_name = $distributor_name,\
                                     d.distributor_org_type = $distributor_org_type,\
                                     d.distributor_address = $distributor_address,\
                                     d.distributor_short_name = $distributor_short_name,\
                                     d.source_identifier = $source_identifier,\
                                     d.start_date = $start_date,\
                                     d.end_date = $end_date,\
                                     d.last_update_date = $last_update_date,\
                                     d.last_update_user = $last_update_user,\
                                     d.version = $version\
                            on match set\
                                     d.country_of_origin = $country_of_origin,\
                                     d.distributor_name = $distributor_name,\
                                     d.distributor_org_type = $distributor_org_type,\
                                     d.distributor_address = $distributor_address,\
                                     d.distributor_short_name = $distributor_short_name,\
                                     d.source_identifier = $source_identifier,\
                                     d.start_date = $start_date,\
                                     d.end_date = $end_date,\
                                     d.last_update_date = $last_update_date,\
                                     d.last_update_user = $last_update_user,\
                                     d.version = $version",
                distributor_id = record['DISTRIBUTOR_ID'],
                distributor_name = record['DISTRIBUTOR_NAME'],
                country_of_origin = record['COUNTRY_OF_ORIGIN'],
                distributor_org_type = record['DISTRIBUTOR_ORG_TYPE'],
                distributor_address = record['DISTRIBUTOR_ADDRESS'],
                distributor_short_name = record['DISTRIBUTOR_SHORT_NAME'],
                source_identifier = record['SOURCE_IDENTIFIER'],
                start_date = record['START_DATE'],
                end_date = record['END_DATE'],
                last_update_date = record['LAST_UPDATE_DT'],
                last_update_user = record['LAST_UPDATE_USER'],
                version = record['VERSION'])

## create hospital node

In [87]:
for record in df_hospital.rdd.collect():
    graph.run("merge (h: Hospital{hospital_id: $hospital_id})\
                            on create set\
                                  h.hospital_address = $hospital_address,\
                                  h.country_of_origin = $country_of_origin,\
                                  h.hospital_name = $hospital_name, \
                                  h.hospital_short_name = $hospital_short_name,\
                                  h.hospital_type = $hospital_type,\
                                  h.source_identifier = $source_identifier,\
                                  h.start_date = $start_date,\
                                  h.end_date = $end_date,\
                                  h.last_update_date = $last_update_date,\
                                  h.last_update_user = $last_update_user,\
                                  h.version = $version\
                            on match set\
                                  h.hospital_address = $hospital_address,\
                                  h.country_of_origin = $country_of_origin,\
                                  h.hospital_name = $hospital_name, \
                                  h.hospital_short_name = $hospital_short_name,\
                                  h.hospital_type = $hospital_type,\
                                  h.source_identifier = $source_identifier,\
                                  h.start_date = $start_date,\
                                  h.end_date = $end_date,\
                                  h.last_update_date = $last_update_date,\
                                  h.last_update_user = $last_update_user,\
                                  h.version = $version",
            hospital_id = record['HOSPITAL_ID'],
            hospital_name = record['HOSPITAL_NAME'],
            country_of_origin = record['COUNTRY_OF_ORIGIN'],
            hospital_type = record['HOSPITAL_TYPE'],
            hospital_short_name = record['HOSPITAL_SHORT_NAME'],
            hospital_address = record['HOSPITAL_ADDRESS'],
            source_identifier = record['SOURCE_IDENTIFIER'],
            start_date = record['START_DATE'],
            end_date = record['END_DATE'],
            last_update_date = record['LAST_UPDATE_DT'],
            last_update_user = record['LAST_UPDATE_USER'],
            version = record['VERSION'])

## create user-role node

In [88]:
for record in df_user_roles.rdd.collect():
    graph.run("merge (u: User_Roles{user_id: $user_id})\
                            on create set\
                                    u.country_of_origin = $country_of_origin,\
                                    u.company_name = $company_name,\
                                    u.designation = $designation,\
                                    u.email_id = $email_id,\
                                    u.first_name = $first_name,\
                                    u.last_name = $last_name,\
                                    u.password = $password,\
                                    u.phone_number = $phone_number,\
                                    u.user_address = $user_address,\
                                    u.user_role = $user_role,\
                                    u.start_date = $start_date,\
                                    u.end_date = $end_date,\
                                    u.last_update_date = $last_update_date,\
                                    u.version = $version\
                            on match set\
                                    u.country_of_origin = $country_of_origin,\
                                    u.company_name = $company_name,\
                                    u.designation = $designation,\
                                    u.email_id = $email_id,\
                                    u.first_name = $first_name,\
                                    u.last_name = $last_name,\
                                    u.password = $password,\
                                    u.phone_number = $phone_number,\
                                    u.user_address = $user_address,\
                                    u.user_role = $user_role,\
                                    u.start_date = $start_date,\
                                    u.end_date = $end_date,\
                                    u.last_update_date = $last_update_date,\
                                    u.version = $version",
             company_name = record['COMPANY_NAME'],
             country_of_origin = record['COUNTRY_OF_ORIGIN'],
             designation = record['DESIGNATION'],
             email_id = record['EMAIL_ID'],
             first_name = record['FIRST_NAME'],
             last_name = record['LAST_NAME'],
             password = record['PASSWORD'],
             phone_number = record['PHONE_NUMBER'],
             user_address = record['USER_ADDRESS'],
             user_id = record['USER_ID'],
             user_role = record['USER_ROLE'],
             start_date = record['START_DATE'],
             end_date = record['END_DATE'],
             last_update_date = record['LAST_UPDATE_DT'],
             version = record['VERSION'])

## create gpo node

In [89]:
for record in df_gpo.rdd.collect():
    graph.run("merge (g: GPO{gpo_id: $gpo_id}) \
                            on create set\
                                    g.gpo_name = $gpo_name,\
                                    g.gpo_short_name = $gpo_short_name, \
                                    g.gpo_org_type = $gpo_org_type, \
                                    g.gpo_address = $gpo_address,\
                                    g.country_of_origin = $country_of_origin,\
                                    g.source_identifier = $source_identifier,\
                                    g.start_date = $start_date,\
                                    g.end_date = $end_date,\
                                    g.last_update_date = $last_update_date,\
                                    g.last_update_user = $last_update_user,\
                                    g.version = $version\
                            on match set\
                                    g.gpo_name = $gpo_name,\
                                    g.gpo_short_name = $gpo_short_name, \
                                    g.gpo_org_type = $gpo_org_type, \
                                    g.gpo_address = $gpo_address,\
                                    g.country_of_origin = $country_of_origin,\
                                    g.source_identifier = $source_identifier,\
                                    g.start_date = $start_date,\
                                    g.end_date = $end_date,\
                                    g.last_update_date = $last_update_date,\
                                    g.last_update_user = $last_update_user,\
                                    g.version = $version",
                gpo_id = record['GPO_ID'], 
                gpo_name = record['GPO_NAME'],
                gpo_short_name = record['GPO_SHORT_NAME'],
                country_of_origin = record['COUNTRY_OF_ORIGIN'],
                gpo_org_type = record['GPO_ORG_TYPE'],
                gpo_address = ['GPO_ADDRESS'],
                source_identifier = record['SOURCE_IDENTIFIER'],
                start_date = record['START_DATE'],
                end_date = record['END_DATE'],
                last_update_date = record['LAST_UPDATE_DT'],
                last_update_user = record['LAST_UPDATE_USER'],
                version = record['VERSION'])

## correlataion node

In [90]:
#df_correlation_data.columns

In [91]:
for record in df_correlation_data.rdd.collect():
    graph.run("merge (n:Correlation_Data{product_id: $product_id,\
                                          distributor_name: $distributor_name,\
                                          distributor_sku: $distributor_sku, \
                                          hospital_name: $hospital_name, \
                                          hospital_sku: $hospital_sku,\
                                          manufacturer_name: $manufacturer_name,\
                                          contract_number: $contract_number,\
                                          contract_price: $contract_price,\
                                          base_price: $base_price})",
                distributor_name = record['DISTRIBUTOR_NAME'],
                distributor_sku = record['DISTRIBUTOR_SKU'],
                hospital_name = record['HOSPITAL_NAME'],
                hospital_sku = record['HOSPITAL_SKU'],
                product_id = record['PRODUCT_ID'],
                manufacturer_name = record['MANUFACTURER'],
                contract_number = record['CONTRACT_NUMBER'],
                contract_price = record['CONTRACT_PRICE'],
                base_price = record['BASE_PRICE'])

## create contract node

In [92]:
# GET CONTRACT DATA FROM CSV
df_contract = spark.read.csv('contract.csv', header = True).dropna(how = 'all')   

for record in df_contract.rdd.collect():
    graph.run("merge (n: Contract{contract_number: $contract_number, \
                                  contract_name: $contract_name, \
                                  contract_description: $contract_description, \
                                  start_date: $start_date, \
                                  end_date: $end_date, \
                                  gpo_name: $gpo_name, \
                                  gpo_sign_date: $gpo_sign_date, \
                                  manufacturer_name: $manufacturer_name, \
                                  manufacurer_sign_date: $manufacurer_sign_date, \
                                  distributor_name: $distributor_name, \
                                  distributor_sign_date: $distributor_sign_date, \
                                  hospital_name: $hospital_name, \
                                  hospital_sign_date: $hospital_sign_date})",
                contract_number = record['CONTRACT_NUMBER'], 
                contract_name = record['CONTRACT_NAME'], 
                contract_description = record['CONTRACT_DESCRIPTION'], 
                start_date = record['START_DATE'], 
                end_date = record['END_DATE'], 
                gpo_name = record['GPO_NAME'], 
                gpo_sign_date = record['GPO_SIGN_DATE'], 
                manufacturer_name = record['MANUFACTURER_NAME'], 
                manufacurer_sign_date = record['MANUFACURER_SIGN_DATE'], 
                distributor_name = record['DISTRIBUTOR_NAME'], 
                distributor_sign_date = record['DISTRIBUTOR_SIGN_DATE'], 
                hospital_name = record['HOSPITAL_NAME'], 
                hospital_sign_date = record['HOSPITAL_SIGN_DATE'])

## create relationship into neo4j

In [93]:
graph.run("match (m: Manufacturer), (p: Product) \
           where p.manufacturer = m.manufacturer_name \
           merge (p) - [r: created_by] -> (m)")

graph.run("match (m: Manufacturer), (d: Distributor), (c: Correlation_Data) \
           where c.manufacturer_name = m.manufacturer_name and c.distributor_name = d.distributor_name \
           merge (m) - [r: supplied_to] -> (d)")

graph.run("match (h: Hospital), (d: Distributor), (c: Correlation_Data) \
           where h.hospital_name = c.hospital_name and c.distributor_name = d.distributor_name \
           merge (d) - [r: shipped_to] -> (h)")

graph.run("match (h: Hospital), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and h.hospital_name = c.hospital_name \
           merge (p) - [r: product_shipped_to{hospital_sku: c.hospital_sku}] -> (h)")

graph.run("match (d: Distributor), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and c.distributor_name = d.distributor_name \
           merge (p) - [r: product_supplied_to{distributor_sku: c.distributor_sku}] -> (d)")

print('COMPLETED')

COMPLETED


In [94]:
graph.run("match (cn: Contract), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and c.contract_number = cn.contract_number \
           merge (p) - [r: in_contract] -> (cn)")   

(No data)